In [3]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
from cfg.configs import Config
from util.args import parse_args
import numpy as np
import scipy.optimize as op
from data.data import SphericalDataset, SphericalBlockDataset, minmax_scale, normalization
from eval.util import mse, mae
# from test import get_model
from time import time
from data.vec_io import *
from data.vis_io import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Rooting Finding: phys -> comp

In [4]:
def deg2rad(deg):
  return deg * np.pi / 180

# for numerical isse
def mysin(a):
  val = np.sin(a)
  val = close_round(val, 0)
  val = close_round(val, 1)
  return val

def mycos(a):
  val = np.cos(a)
  val = close_round(val, 0)
  val = close_round(val, 1)
  return val

def close_round(val, test_val, abs_bounds=1e-12):
  isclose = np.abs(test_val - val) < abs_bounds
  # print(isclose)
  if isinstance(val, float) or isinstance(val, int) or np.float32:
    val_cp = test_val if isclose else val
  else:
    val_cp = val.copy()
    val_cp[isclose] = test_val
  return val_cp

def sph2car(r, theta, phi):
  # x_coef = np.sin(phi)*np.cos(theta)
  # y_coef = np.around(np.sin(phi)*np.sin(theta), decimals=10)
  # z_coef = np.around(np.cos(phi), decimals=10)
  x = r*mysin(phi)*mycos(theta)
  y = r*mysin(phi)*mysin(theta)
  z = r*mycos(phi)
  
  return np.array([x,y,z])

def car2sph(x, y, z):
  # assert (x or y) != 0
  r = np.sqrt(x*x + y*y + z*z)
  theta = np.arctan(y/x)
  phi = np.arctan(np.sqrt(x*x + y*y) / z)
  # if x > 0:
  #   phi = np.arctan(y/x)
  # elif x < 0 and y >= 0:
  #   phi = np.arctan(y/x)+np.pi
  # elif x < 0 and y < 0:
  #   phi == np.arctan(y/x)-np.pi
  # elif x == 0 and y > 0:
  #   phi = np.pi/2
  # elif x == 0 and y < 0:
  #   phi = -np.pi/2

  return np.array([r,theta,phi])
# //
# //		    6________7  high-vtx
# //		   /|       /|
# //		  / |      / |
# //		4/_______5/  |
# //		|  2|___ |___|3
# //		|  /     |  /
# //		| /      | /
# //		|/_______|/
# //		0        1
# //  low_vtx

# //
# //		 011_________111  high-vtx
# //		   /|       /|
# //		  / |      / |
# //	001/_____101/  |
# //		|010|___ |___|110
# //		|  /     |  /
# //		| /      | /
# //		|/_______|/
# //	000       100


def trilerp(coord: np.ndarray, low_vtx: np.ndarray, high_vtx: np.ndarray, cell_val: np.ndarray):
  # x0,y0,z0 = low_vtx
  # x1,y1,z1 = high_vtx
  # x, y, z = coord

  # low_vtx, high_vtx, coord = normalization(np.array([low_vtx, high_vtx, coord]), new_min=0, dim=0)

  # TODO: Why trilerp doesn't work if low and high are not normalized to 1
  x, y, z = (coord - low_vtx) / (high_vtx - low_vtx)
  low_vtx = np.zeros(3)
  high_vtx = np.ones(3)
  x0,y0,z0 = low_vtx
  x1,y1,z1 = high_vtx

  v000 = cell_val[0,0,0]
  v100 = cell_val[1,0,0]
  v010 = cell_val[0,1,0]
  v110 = cell_val[1,1,0]
  v001 = cell_val[0,0,1]
  v101 = cell_val[1,0,1]
  v011 = cell_val[0,1,1]
  v111 = cell_val[1,1,1]
  # print(x, y, z)
  # print(x0, y0, z0)
  # print(x1, y1, z1)
  # print((x0-x1)*(y0-y1)*(z0-z1))
  # print(((x0-x1)*(y0-y1)*(z0-z1)))
  # for cord in [v000, v100, v010, v110, v001, v101, v011, v111]:
  #   print(cord)
  a0 = (
    (-v000*x1*y1*z1 + v001*x1*y1*z0 + v010*x1*y0*z1 - v011*x1*y0*z0 + 
    v100*x0*y1*z1 - v101*x0*y1*z0 - v110*x0*y0*z1 + v111*x0*y0*z0)
  )
  a1 = (
    (v000*z1*y1 - v001*z0*y1 - v010*z1*y0 + v011*z0*y0 - 
    v100*z1*y1 + v101*z0*y1 + v110*z1*y0 - v111*z0*y0)
  )
  a2 = (
    (v000*x1*z1 - v001*x1*z0 - v010*x1*z1 + v011*x1*z0 - 
    v100*x0*z1 + v101*x0*z0 + v110*x0*z1 - v111*x0*z0)
  )
  a3 = (
    (v000*x1*y1 - v001*x1*y1 - v010*x1*y0 + v011*x1*y0 - 
    v100*x0*y1 + v101*x0*y1 + v110*x0*y0 - v111*x0*y0)
  )
  a4 = (
    (-v000*z1 + v001*z0 + v010*z1 - v011*z0 + 
    v100*z1 - v101*z0 - v110*z1 + v111*z0)
  )
  a5 = (
    (-v000*y1 + v001*y1 + v010*y0 - v011*y0 + 
    v100*y1 - v101*y1 - v110*y0 + v111*y0)
  )
  a6 = (
    (-v000*x1 + v001*x1 + v010*x1 - v011*x1 + 
    v100*x0 - v101*x0 - v110*x0 + v111*x0)
  )
  a7 = (
    (v000 - v001 - v010 + v011 -v100 + v101 + v110 - v111)
  )
  interpolant = (a0 + a1*x + a2*y + a3*z + a4*x*y + a5*x*z + a6*y*z + a7*x*y*z) / ((x0-x1)*(y0-y1)*(z0-z1))
  coeff = np.array([a0, a1, a2, a3, a4, a5, a6, a7]).T
  return interpolant, coeff

# G
def comp2phys(comp: np.ndarray, vtx_low, vtx_high, cell_coords):
  phys_est_x, a = trilerp(comp, vtx_low, vtx_high, cell_coords[..., 0])
  phys_est_y, b = trilerp(comp, vtx_low, vtx_high, cell_coords[..., 1])
  phys_est_z, c = trilerp(comp, vtx_low, vtx_high, cell_coords[..., 2])
  return np.array([phys_est_x, phys_est_y, phys_est_z]), np.array([a,b,c])

# F
def diff_phys(comp: np.ndarray, phys: np.ndarray, vtx_low, vtx_high, cell_coords):
  phys_est, coeff = comp2phys(comp, vtx_low, vtx_high, cell_coords)
  return phys - phys_est

def diff_phys_dim(comp: np.ndarray, phys: np.ndarray, vtx_low, vtx_high, cell_coords_dim):
  phys_est = trilerp(comp, vtx_low, vtx_high, cell_coords_dim)
  return phys - phys_est
# op.root_scalar(diff_phys, method='newton')

def diff_phys_dim_inv_jac():
  pass

class CurviInterpField:
  def __init__(self):
    pass

class CurviInterpCell:
  def __init__(self, min_comp, max_comp, phys_coords):
    self.min_comp = np.array(min_comp)
    self.max_comp = np.array(max_comp)
    self.phys = phys_coords
    self.init_comp = min_comp + (max_comp - min_comp) / 2
    self.coeff = None

    # xs = [min_comp[0], max_comp[0]]
    # ys = [min_comp[1], max_comp[1]]
    # zs = [min_comp[2], max_comp[2]]
    # self.interp = RegularGridInterpolator((xs, ys, zs), phys_coords)

  # Newton's method to find computational coordinate given physical cooridnate
  def phys2comp(self, phys: np.ndarray, init_comp=None, tol=1.48e-8, maxiter=50, rtol=0.0):
    if init_comp is None:
      init_comp = self.init_comp

    comp = init_comp
    goal_diff = np.zeros(3)
    for i in range(maxiter):
      phys_est, coeff = self.comp2phys(comp)
      diff_funcval = (phys_est - phys)

      jac_inv = self.get_jacobian_inv(comp, coeff)

      new_comp = comp - (jac_inv @ diff_funcval.reshape(3, 1)).flatten()
      # print("My Jac inv\n", jac_inv, diff_funcval, "\n")
      # print(phys_est.shape, coeff.shape, phys.shape)
      # print(new_comp.shape, comp.shape)
      # print(jac_inv.shape, diff_funcval.shape)
      # print(f'coeff: {coeff[:, 0]}')
      print(f'phys_est: {phys_est} comp: {new_comp}  error: {np.abs(diff_funcval-goal_diff)}')
      # if np.all(np.isclose(new_comp, comp, rtol=rtol, atol=tol)):
      if np.all(np.isclose(diff_funcval, goal_diff, rtol=rtol, atol=tol)):
        print(f'iter {i}')
        return new_comp
      comp = new_comp
    return comp

  # def phys2comp(self, phys):
  #   return self.find_comp(phys)

  def comp2phys(self, comp):
    phys_est, coeff = trilerp(comp, self.min_comp, self.max_comp, self.phys)
    # phys_est_y, b = trilerp(comp, self.min_comp, self.max_comp, self.phys[..., 1])
    # phys_est_z, c = trilerp(comp, self.min_comp, self.max_comp, self.phys[..., 2])
    # self.coeff = np.array([a,b,c])
    # return np.array([phys_est_x, phys_est_y, phys_est_z]), np.array([a,b,c])
    # print(phys_est.shape, coeff.shape)
    # self.interp(comp)
    return phys_est, coeff
  
  def get_jacobian(self, comp, coeff):
    a,b,c = coeff
    # print("COE: \n", coeff.T)
    # print("My COMP: \n", comp)
    return np.array([
      [
        a[1] + a[4]*comp[1] + a[5]*comp[2] + a[7]*comp[1]*comp[2],
        a[2] + a[4]*comp[0] + a[6]*comp[2] + a[7]*comp[0]*comp[2],
        a[3] + a[5]*comp[0] + a[6]*comp[1] + a[7]*comp[0]*comp[1],
      ],
      [
        b[1] + b[4]*comp[1] + b[5]*comp[2] + b[7]*comp[1]*comp[2],
        b[2] + b[4]*comp[0] + b[6]*comp[2] + b[7]*comp[0]*comp[2],
        b[3] + b[5]*comp[0] + b[6]*comp[1] + b[7]*comp[0]*comp[1],
      ],
      [
        c[1] + c[4]*comp[1] + c[5]*comp[2] + c[7]*comp[1]*comp[2],
        c[2] + c[4]*comp[0] + c[6]*comp[2] + c[7]*comp[0]*comp[2],
        c[3] + c[5]*comp[0] + c[6]*comp[1] + c[7]*comp[0]*comp[1],
      ]
    ])
  
  def get_jacobian_inv(self, comp, coeff):
    jac = self.get_jacobian(comp, coeff)
    # np_det = np.linalg.det(jac)
    # np_inv = np.linalg.inv(jac)
    det = (-jac[0,0]*jac[1,1]*jac[2,2] - jac[0,1]*jac[1,2]*jac[2,0] - jac[0,2]*jac[1,0]*jac[2,1] + 
            jac[0,2]*jac[1,1]*jac[2,0] + jac[0,1]*jac[1,0]*jac[2,2] + jac[0,0]*jac[1,2]*jac[2,1])
    inv = np.array([
      [
        jac[1,1]*jac[2,2] - jac[1,2]*jac[2,1],
        -jac[0,1]*jac[2,2] + jac[0,2]*jac[2,1],
        jac[0,1]*jac[1,2] - jac[0,2]*jac[1,1],
      ],
      [
        -jac[1,0]*jac[2,2] + jac[1,2]*jac[2,0],
        jac[0,0]*jac[2,2] - jac[0,2]*jac[2,0],
        -jac[0,0]*jac[1,2] + jac[0,2]*jac[1,0],
      ],
      [
        jac[1,0]*jac[2,1] - jac[1,1]*jac[2,0],
        jac[0,0]*jac[2,1] - jac[0,1]*jac[2,0],
        jac[0,0]*jac[1,1] - jac[0,1]*jac[1,0],
      ],
    ])
    # print(jac)
    # print(np_inv)
    # print(inv/det)
    # print(np_det)
    # print(det)
    # print("My Jac inv\n", jac, "\n", inv, det, "\n")
    return inv/det

In [87]:
cic = CurviInterpCell(min_comp=low_vtx, max_comp=high_vtx, phys_coords=phys)
print(f"Testing point in phys is { point_phys}")
print(f"Testing point in comp GT is { point}")
print(f"Testing point initial comp is {init_guess}")
print(f"Testing point initial phys_est is {cic.comp2phys(init_guess)[0]}\n\n")
cic = CurviInterpCell(min_comp=np.zeros(3), max_comp=np.ones(3), phys_coords=phys)
comp_est_newton = cic.phys2comp(point_phys, maxiter=50, tol=1e-4)

Testing point in phys is [ -768.3575  4154.1406 -2542.9346]
Testing point in comp GT is [4930.9       100.4791    121.045204]
Testing point initial comp is [4935.25     100.77994  121.3485 ]
Testing point initial phys_est is [ -788.2636  4140.097  -2567.4258]


phys_est: [ -788.26385  4140.097   -2567.4253 ] comp: [0.21455218 0.19928449 0.36448123]  error: [19.90637207 14.04345703 24.49072266]
phys_est: [ -766.9464  4146.802  -2555.2021] comp: [0.21458255 0.20000063 0.28526197]  error: [ 1.41107178  7.33886719 12.26757812]
phys_est: [ -767.6507  4150.319  -2549.316 ] comp: [0.21454412 0.19999933 0.24400584]  error: [0.70678711 3.82177734 6.38134766]
phys_est: [ -767.99005  4152.1543  -2546.25   ] comp: [0.21456843 0.1999998  0.22254731]  error: [0.36743164 1.98632812 3.31542969]
phys_est: [ -768.1668  4153.1104 -2544.6555] comp: [0.21453571 0.20000129 0.2113968 ]  error: [0.19067383 1.03027344 1.72094727]
phys_est: [ -768.25867  4153.606   -2543.8267 ] comp: [0.21455341 0.20000026 0.20

In [5]:
bds = SphericalBlockDataset('data/earth_nopole.npy', [0,1,2], [3,4,5], downscale=2, intrans=None, outtrans=None)
for curv_block in bds.curvs:
  # curv_block = pp.inverse_transform(curv_block)
  print(curv_block.shape, curv_block.min(0)[0], curv_block.max(0)[0])

KeyboardInterrupt: 

In [5]:
print(bds.block_indices.shape)
print(bds.block_indices[0])

(8, 3, 2)
[[  0 100]
 [  0 180]
 [  0  80]]


In [6]:
np.array([5, 105, 90]) in bds.block_indices[0]

/tmp/ipykernel_1371/1899325734.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.array([5, 105, 90]) in bds.block_indices[0]


False

In [ ]:
ds = SphericalDataset('data/earth_nopole.npy', [0,1,2], [3,4,5])
ds.curv.shape

(201, 360, 160, 3)

In [ ]:
xyz = read_vec('../test/src/data/mantle_phys.vec')
xg, yg, zg = read_grid('../test/src/data/mantle_comp.grid')
xyz_comp = get_mesh(xg, yg, zg)
ds.cart = xyz
ds.curv = xyz_comp

meshe generated: (201, 360, 140, 3)


In [ ]:
phys = np.array(ds.cart[100:102, 100:102, 100:102])
comp = np.array(ds.curv[100:102, 100:102, 100:102])
# phi dimension is decressing, which breaks trilerp implementation
# comp = np.flip(comp, 2)
# phys_calc = np.array([sph2car(sph[0], *np.radians(sph[1:])) for sph in comp.reshape(-1, 3)]).reshape(2,2,2,3)


# get the mean coordinate as initial guess
init_guess = comp.reshape(-1, 3).mean(0)

init_guess

low_vtx = comp[0,0,0]
high_vtx = comp[1,1,1]
xs = [low_vtx[0], high_vtx[0]]
ys = [low_vtx[1], high_vtx[1]]
zs = [low_vtx[2], high_vtx[2]]

cell_coords_x, cell_coords_y, cell_coords_z = np.meshgrid(
  xs, ys, zs, indexing='ij',  sparse=False
)


point = low_vtx + (high_vtx - low_vtx)*0.2
display(low_vtx, high_vtx)

point_phys = sph2car(point[0], *np.radians(point[1:]))
print(f"Testing point in phys is { point_phys}")
print(f"Testing point in comp GT is { point}")


array([4928.     ,  100.27855,  120.843  ], dtype=float32)

array([4942.5     ,  101.281334,  121.854   ], dtype=float32)

Testing point in phys is [ -768.3575  4154.1406 -2542.9346]
Testing point in comp GT is [4930.9       100.4791    121.045204]


In [33]:
cfgObj = Config('nntest.yaml')
nncfg = Config('nn.yaml')
testcfg = cfgObj.config
# get model
model_path = testcfg['model_path']
model = get_model(model_path, nncfg)

In [77]:
low_vtx = [0, 0]
high_vtx = [1, 1]
cic = CurviInterpCell(min_comp=low_vtx, max_comp=high_vtx, phys_coords=phys)
print(f"Testing point in phys is { point_phys}")
print(f"Testing point in comp GT is { point}")
print(f"Testing point initial phys_est is {cic.comp2phys(init_guess)[0]}\n\n")
st = time()
cic = CurviInterpCell(min_comp=low_vtx, max_comp=high_vtx, phys_coords=phys)
for i in range(100):
  comp_est_newton = cic.phys2comp(point_phys, maxiter=10, tol=1e-4)
t_newton = time() - st

# st = time()
# for i in range(100):
#   point_phys_nn = ds.inpp.transform(point_phys)
#   with torch.no_grad():
#     comp_est_nn = model(torch.Tensor(point_phys))
#   comp_est_nn = ds.outpp.inverse_transform(comp_est_nn.numpy())
# t_nn = time() - st

# print(f"newton prediction: {comp_est_newton}, error: {mse(comp_est_newton, point)}, time: {t_newton}")
# print(f"NN     prediction: {comp_est_nn}, error: {mse(comp_est_nn, point)}, time: {t_nn}")

Testing point in phys is [ -768.3575  4154.1406 -2542.9346]
Testing point in comp GT is [4930.9       100.4791    121.045204]
Testing point initial phys_est is [ -788.2636  4140.097  -2567.4258]


newton prediction: [4930.9214   100.47142  121.01765], error: 0.00042667085654102266, time: 0.45264220237731934
NN     prediction: [6370.0264   182.2747   189.24663], error: 694142.3125, time: 0.2562694549560547


In [ ]:
cell_coords = np.concatenate([cell_coords_x[...,None], cell_coords_y[...,None], cell_coords_z[...,None]], -1)

In [21]:
comp2phy(point, low_vtx, high_vtx, cell_coords)

4928.0 4942.5 4928.0 4942.5 4928.0 4942.5 4928.0 4942.5
100.27855 100.27855 101.281334 101.281334 100.27855 100.27855 101.281334 101.281334
69.27 69.27 69.27 69.27 68.258 68.258 68.258 68.258


(array([5361.5796  ,  136.56906 ,   57.702682], dtype=float32),
 array([[   0.,   16.,    0.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,   20.,    0.,    0.,    0.,    0.,    0.],
        [-256.,    0.,    0.,   16.,    0.,    0.,    0.,    0.]],
       dtype=float32))

In [18]:
sol = op.root(diff_phys_dim, x0=init_guess[0], args=(point_phys, low_vtx, high_vtx, cell_coords_x))

ValueError: not enough values to unpack (expected 3, got 1)

In [238]:
init_guess

array([4935.25   ,  100.77994,   68.764  ], dtype=float32)

In [237]:
sol.x

array([    1671.23568275, -1353139.43757878,  -336520.7910641 ])

In [85]:
xg, yg ,zg = np.meshgrid([0,1], [0,1], [0,1],indexing='ij',  sparse=True)
def f(x,y,z):
  return x + 10*y + 100*z
# my lerp
coord = [0.215, 0.282, 0.2162]
low_vtx = [-1,-2,-3]
high_vtx = [1,2,3]
cell_val = f(xg, yg, zg)
interpolant = trilerp(coord, low_vtx, high_vtx, cell_val)
gt = f(*coord)
print(f"gt: {gt}    -      lerp: {interpolant}")

0.215 0.282 0.2162
-1 -2 -3
1 2 3
-48
-48
0
1
10
11
100
101
110
111
gt: 24.655    -      lerp: (59.91583333333333, array([-2664,   -24,  -120,  -800,     0,     0,     0,     0]))


In [91]:
comp[...,1][0,:,0]

array([100.27855 , 101.281334], dtype=float32)

In [ ]:
diff_phys(phys, comp, )

In [84]:
# SCIPY lerp
from scipy.interpolate import RegularGridInterpolator

# x = np.arange(2)
# y = np.arange(2)
# z = np.arange(2)
# xg, yg ,zg = np.meshgrid(x, y, z, indexing='ij', sparse=True)
# data = f(xg, yg, zg)

x= comp[...,0][:,0,0]
y= comp[...,1][0,:,0]
z= comp[...,2][0,0,:]

xs = [low_vtx[0], high_vtx[0]]
ys = [low_vtx[1], high_vtx[1]]
zs = [low_vtx[2], high_vtx[2]]

my_interpolating_function = RegularGridInterpolator((xs, ys, zs), cell_val)
my_interpolating_function(coord)

array([59.91583333])